# More on classes

We begin with review and design considerations.

In [17]:
class Widget: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [18]:
Widget('10 years', '10 pounds')

Widget('10 years', '10 pounds')

In [19]:
Widget(10.0, 1.0) == Widget(10, 1)

True

In [20]:
Widget(10.0, 1.0) == Widget(10, 2)

False

In [21]:
class Employee: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [26]:
Employee(23.0, 200.0) == Employee(23, 200)

True

In [25]:
Employee(23.0, 200.0) == Employee(20, 200)

False

In [27]:
w = Widget(70, 180)

In [28]:
w == 42

False

In [29]:
e = Employee(70, 180)

In [30]:
w == e

False

In [31]:
class ExplodingWidget(Widget):
    def explode(self):
        print(f'{self!r} has exploded!')

In [32]:
ew = ExplodingWidget(70, 180)

In [33]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [34]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [35]:
w == ew

True

In [36]:
w.__dict__ == e.__dict__

True